In [17]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [18]:
def segmentation(width,height,inputImage,filename):
    start=0
    end=0
    j=0
    
    for row in range(width):
        #print(vertical_projection[row])
        #print(horizontal_projection[row+1])
        if vertical_projection[row-1]==0 and vertical_projection[row]!=0 : 
        
            start=row-1
            #print("ss",start)
            #print(vertical_projection[row])
        if row+1<width:
            if vertical_projection[row-1]!=0 and vertical_projection[row]==0 :
                end=row+1
            
                #print("ee",end)
                #print(vertical_projection[row-1])
                
                if(end-1-start>=7):
                    if not os.path.exists("./caracteres/"+filename.lower().replace(".png", "", 1)+"/"):
                        os.mkdir("./caracteres/"+filename.lower().replace(".png", "", 1)+"/")
                        #print("s",start)
                        #print("e",end)
                        roi=inputImage[0:height,start:end]
                        h,w,c=roi.shape
                        #print(h,w)
                    
                        cv2.imwrite("./caracteres/"+filename.lower().replace(".png", "", 1)+"/caractere"+str(j)+".png",roi)
                        j=j+1
                        start=0
                        end=0
                        #plt.imshow(roi)
                    else:
                        roi=inputImage[0:height,start:end]
                        h,w,c=roi.shape
                        #print(h,w)
                    
                        cv2.imwrite("./caracteres/"+filename.lower().replace(".png", "", 1)+"/caractere"+str(j)+".png",roi)
                        j=j+1
                        start=0
                        end=0
                
        #print(start,ed)
        if start!=0 and end==0 and row==width-1:
            roi=inputImage[0:height,start:width]
            h,w,c=roi.shape
            #print(h,w)
            #print(filename)
            #print(filename.lower().replace(".png", "", 1))
            if not os.path.exists("./caracteres/"+filename.lower().replace(".png", "", 1)+"/"):
                os.mkdir("./caracteres/"+filename.lower().replace(".png", "", 1)+"/")
                
                cv2.imwrite("./caracteres/"+filename.lower().replace(".png", "", 1)+"/caractere"+str(j)+".png",roi)
                j=j+1
                #plt.imshow(roi)
                start=0
                end=0
                
            else:
                cv2.imwrite("./caracteres/"+filename.lower().replace(".png", "", 1)+"/caractere"+str(j)+".png",roi)
                j=j+1
                #plt.imshow(roi)
                start=0
                end=0
        
                
            
        

In [19]:
path="./words/"
for filename in os.listdir(path):
    im_gray = cv2.imread(path+filename,  cv2.IMREAD_GRAYSCALE)
    h,w=im_gray .shape
    image = cv2.GaussianBlur(im_gray, (5,5), 1)
    th =  cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,2)
    #plt.imshow(th)
    cv2.imwrite('img.jpg',th)
    image=cv2.imread('img.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # create an empty mask
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    #plt.imshow(mask)
    # loop through the contours
    for i, cnt in enumerate(contours):
        # if the contour has no other contours inside of it
        if hierarchy[0][i][2] == -1:
            # if the size of the contour is greater than a threshold
            if cv2.contourArea(cnt) <70:
                cv2.drawContours(mask, [cnt], 0, (255), -1)
                #plt.imshow(mask)
        #result = binarizedImage - (mask+255)
    result=th+mask
    result = cv2.GaussianBlur(result, (3,3), 0)
    _, binarizedImage = cv2.threshold(result, 127, 255, cv2.THRESH_BINARY)
    binarizedImage[binarizedImage == 0] = 1
    binarizedImage[binarizedImage == 255] = 0
    vertical_projection = np.sum(binarizedImage, axis=0)
    for i in range(len(vertical_projection)):
        if(vertical_projection[i]<=4):
            vertical_projection[i]=0
    segmentation(w,h,image,filename)